In [49]:
### Required Packages
from termcolor import colored
import math
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from datetime import datetime
import pickle as pkl
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
import tqdm
import time
import transformers
from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline
use_cuda = torch.cuda.is_available()
import transformers
from transformers import BertForSequenceClassification
from collections import OrderedDict

In [2]:
file_path = "../processed_after.csv"
df = pd.read_csv(file_path)

In [3]:
df.columns

Index(['patient_id', 'admission_date', 'discharge_date', 'Medical_code', 'poa',
       'diagnosis_priority', 'Billing_source'],
      dtype='object')

In [4]:
df

,patient_id,admission_date,discharge_date,Medical_code,poa,diagnosis_priority,Billing_source
0,6338,1996-02-14,1996-02-14,ICD9_027.1,0,1,0
1,35880,1996-02-27,1996-02-27,ICD9_031.9,0,1,0
2,12074,1996-02-16,1996-02-16,ICD9_100.9,0,1,0
3,13430,1996-02-27,1996-02-27,ICD9_031.2,0,1,0
4,39302,1996-02-16,1996-02-16,ICD9_027.1,0,1,0
...,...,...,...,...,...,...,...
10861332,8672,2014-01-07,2014-01-07,ICD9_626.1,0,1,0
10861333,24564,2014-01-15,2014-01-15,ICD9_536.8,0,1,0
10861334,14970,2014-01-08,2014-01-08,ICD9_787.3,0,1,0
10861335,3127,2014-01-07,2014-01-07,ICD9_786.2,0,1,0


In [5]:
df = df.drop_duplicates()

In [6]:
vocab = pkl.load(open("../medbert.types", 'rb'), encoding='bytes')

In [7]:
total_patient = set(df.patient_id)
len(total_patient)

39831

In [8]:
# "438.53","ate effects of cerebrovascular disease, other paralytic syndrome, bilateral","雙側之其他癱瘓徵候群，腦血管疾病後遺症"
data_438_53 = df.loc[df['Medical_code']=='ICD9_438.53']
print(len(data_438_53))

200


In [9]:
# "433.00","cclusion and stenosis of basilar artery without mention of cerebral infarction","未提及腦梗塞之基底動脈阻塞及狹窄"
data_433_00 = df.loc[df['Medical_code']=='ICD9_433.00']
print(len(data_433_00))

250


In [10]:
# "433.01","cclusion and stenosis of basilar artery with cerebral infarction","伴有腦梗塞之基底動脈阻塞及狹窄"
data_433_01 = df.loc[df['Medical_code']=='ICD9_433.01']
print(len(data_433_01))

92


In [11]:
# "433.10","cclusion and stenosis of carotid artery without mention of cerebral infarction","未提及腦梗塞之頸動脈阻塞及狹窄"
data_433_10 = df.loc[df['Medical_code']=='ICD9_433.10']
print(len(data_433_10))

649


In [12]:
# "433.11","cclusion and stenosis of carotid artery with cerebral infarction","伴有腦梗塞之頸動脈阻塞及狹窄"
data_433_11 = df.loc[df['Medical_code']=='ICD9_433.11']
print(len(data_433_11))

161


In [13]:
# "433.20","cclusion and stenosis of vertebral artery without mention of cerebral infarction","未提及腦梗塞之椎動脈阻塞及狹窄"
data_433_20 = df.loc[df['Medical_code']=='ICD9_433.20']
print(len(data_433_20))

324


In [14]:
# "433.21","cclusion and stenosis of vertebral artery with cerebral infarction","伴有腦梗塞之椎動脈阻塞及狹窄"
data_433_21 = df.loc[df['Medical_code']=='ICD9_433.21']
print(len(data_433_21))

88


In [15]:
# "433.30","cclusion and stenosis of multiple and bilateral precerebral arteries without mention of cerebral infarction","未提及腦梗塞之多發性及兩側腦前動脈阻塞及狹窄"
data_433_30 = df.loc[df['Medical_code']=='ICD9_433.30']
print(len(data_433_30))

1


In [16]:
# "433.31","cclusion and stenosis of multiple and bilateral precerebral arteries with cerebral infarction","伴有腦梗塞之多發性及兩側腦前動脈阻塞及狹窄"
data_433_31 = df.loc[df['Medical_code']=='ICD9_433.31']
print(len(data_433_31))

45


In [17]:
# "433.80","cclusion and stenosis of other specified precerebral artery without mention of cerebral infarction","未提及腦梗塞之其他特定之腦前動脈阻塞及狹窄"
data_433_80 = df.loc[df['Medical_code']=='ICD9_433.80']
print(len(data_433_80))

14


In [18]:
# "433.81","cclusion and stenosis of other specified precerebral artery with cerebral infarction","伴有腦梗塞之其他特定之腦前動脈阻塞及狹窄"
data_433_81 = df.loc[df['Medical_code']=='ICD9_433.81']
print(len(data_433_81))

3


In [19]:
# "433.90","cclusion and stenosis of unspecified precerebral artery without mention of cerebral infarction","未提及腦梗塞之腦前動脈阻塞及狹窄"
data_433_90 = df.loc[df['Medical_code']=='ICD9_433.90']
print(len(data_433_90))

247


In [20]:
# "433.91","cclusion and stenosis of unspecified precerebral artery with cerebral infarction","伴有腦梗塞之腦前動脈阻塞及狹窄"
data_433_91 = df.loc[df['Medical_code']=='ICD9_433.91']
print(len(data_433_91))
# ==============================================================================================================================

14


In [8]:
patient = set()

In [9]:
# "997.02","atrogenic cerebrovascular infarction or hemorrhage","腦血管梗塞或出血"
data_997_02 = df.loc[df['Medical_code']=='ICD9_997.02']
print(len(data_997_02))

35


In [10]:
for k in data_997_02.index:
    patient.add(data_997_02.loc[k, "patient_id"])

In [11]:
# "434.00","erebral thrombosis without mention of cerebral infarction","未提及腦梗塞之腦血栓症"
data_434_00 = df.loc[df['Medical_code']=='ICD9_434.00']
print(len(data_434_00))

1724


In [12]:
# "434.01","erebral thrombosis with cerebral infarction","腦血栓症合併腦梗塞"
data_434_01 = df.loc[df['Medical_code']=='ICD9_434.01']
print(len(data_434_01))

3915


In [13]:
for k in data_434_00.index:
    patient.add(data_434_00.loc[k, "patient_id"])
for k in data_434_01.index:
    patient.add(data_434_01.loc[k, "patient_id"])

In [14]:
# "434.10","erebral embolism without mention of cerebral infarction","未提及腦梗塞之腦栓塞症"
data_434_10 = df.loc[df['Medical_code']=='ICD9_434.10']
print(len(data_434_10))

465


In [15]:
# "434.11","erebral embolism with cerebral infarction","伴有腦梗塞之腦栓塞症"
data_434_11 = df.loc[df['Medical_code']=='ICD9_434.11']
print(len(data_434_11))

565


In [16]:
for k in data_434_10.index:
    patient.add(data_434_10.loc[k, "patient_id"])
for k in data_434_11.index:
    patient.add(data_434_11.loc[k, "patient_id"])

In [17]:
# "434.90","erebral artery occlusion, unspecified without mention of cerebral infarction","未提及腦梗塞之腦動脈阻塞"
data_434_90 = df.loc[df['Medical_code']=='ICD9_434.90']
print(len(data_434_90))

3301


In [18]:
# "434.91","erebral artery occlusion, unspecified with cerebral infarction","伴有腦梗塞之腦動脈阻塞"
data_434_91 = df.loc[df['Medical_code']=='ICD9_434.91']
print(len(data_434_91))

5818


In [19]:
for k in data_434_90.index:
    patient.add(data_434_90.loc[k, "patient_id"])
for k in data_434_91.index:
    patient.add(data_434_91.loc[k, "patient_id"])

In [20]:
print(len(patient))

951


In [21]:
control_patient = random.sample(total_patient.difference(patient), 2000)

In [22]:
control_data = df.loc[(df["patient_id"].isin(control_patient)) & (df["Medical_code"].isin(vocab.keys()))]
control_data = control_data.sort_values(by=['patient_id', 'admission_date', 'diagnosis_priority']).reset_index()
control_data 

,index,patient_id,admission_date,discharge_date,Medical_code,poa,diagnosis_priority,Billing_source
0,55994,10,1996-04-29,1996-04-29,ICD9_090.9,0,1,0
1,55995,10,1996-04-29,1996-04-29,ICD9_039.1,0,2,0
2,69485,10,1996-05-16,1996-05-16,ICD9_031.2,0,1,0
3,104348,10,1996-06-08,1996-06-08,ICD9_520.8,0,1,0
4,91332,10,1996-07-20,1996-07-20,ICD9_031.2,0,1,0
...,...,...,...,...,...,...,...,...
419617,10363431,39800,2013-08-07,2013-08-07,ICD9_705.81,0,2,0
419618,10363426,39800,2013-08-09,2013-08-09,ICD9_465.9,0,1,0
419619,10363427,39800,2013-08-09,2013-08-09,ICD9_466.0,0,2,0
419620,10762986,39800,2013-08-12,2013-08-12,ICD9_521.0,0,1,0


In [23]:
eligible_data = df.loc[df["patient_id"].isin(patient) & df["Medical_code"].isin(vocab.keys())]
eligible_data = eligible_data.sort_values(by=['patient_id', 'admission_date', 'diagnosis_priority'])
eligible_data = eligible_data.reset_index()

In [24]:
eligible_data

,index,patient_id,admission_date,discharge_date,Medical_code,poa,diagnosis_priority,Billing_source
0,437826,19,1997-11-08,1997-11-08,ICD9_039.9,0,1,0
1,437827,19,1997-11-08,1997-11-08,ICD9_039.1,0,2,0
2,555857,19,1998-01-12,1998-01-12,ICD9_031.2,0,1,0
3,557141,19,1998-02-07,1998-02-07,ICD9_031.2,0,1,0
4,801638,19,1998-02-16,1998-02-16,ICD9_521.0,0,1,0
...,...,...,...,...,...,...,...,...
488276,10383724,39787,2013-08-09,2013-08-09,ICD9_708.8,0,1,0
488277,10383723,39787,2013-08-09,2013-08-09,ICD9_276.1,0,2,0
488278,10383725,39787,2013-08-09,2013-08-09,ICD9_694.4,0,2,0
488279,10383726,39787,2013-08-09,2013-08-09,ICD9_401.9,0,3,0


In [69]:
pt = OrderedDict(eligible_data.groupby(['patient_id']).size().reset_index(name='count').sort_values(by=['count']).reset_index()['patient_id'])

In [70]:
pt

OrderedDict([(0, 15466),
             (1, 27513),
             (2, 16253),
             (3, 14104),
             (4, 7624),
             (5, 16834),
             (6, 9530),
             (7, 3785),
             (8, 8192),
             (9, 31365),
             (10, 37260),
             (11, 4927),
             (12, 30275),
             (13, 35938),
             (14, 25404),
             (15, 26527),
             (16, 25474),
             (17, 21055),
             (18, 27621),
             (19, 5778),
             (20, 18089),
             (21, 21932),
             (22, 31688),
             (23, 31023),
             (24, 22584),
             (25, 14362),
             (26, 8020),
             (27, 12762),
             (28, 32788),
             (29, 12872),
             (30, 4361),
             (31, 13656),
             (32, 30735),
             (33, 31659),
             (34, 10092),
             (35, 21971),
             (36, 8440),
             (37, 6485),
             (38, 4261),
       

In [27]:
# eligible_data.drop_duplicates()
eligible_data.shape[0]

488281

In [28]:
tt_icd = set(df["Medical_code"])
len(tt_icd)

7225

In [29]:
len(vocab.keys())

7226

In [30]:
# print(vocab)

In [31]:
type((df.loc[0, "patient_id"]))
# type(vocab["ICD9_039.9"])

numpy.int64

## Pickled list
The pickled list is a list of lists where each sublist represent a patient record that looks like [pt_id,label, seq_list , segment_list ] where Label: 1: pt developed HF (case) , 0 control seq_list: list of all medical codes in all visits segment list: the visit number mapping to each code in the sequence list

In [32]:
def preprocess(pkList, data, label):
    k = 0
    while(True):
        seq_list, segment_list = [], []
        pt_id = data.loc[k, "patient_id"]
        dt = data.loc[k, "admission_date"]
        dt_idx = 1
        seq_list.append(vocab[data.loc[k, "Medical_code"]])
        segment_list.append(dt_idx)

        while(data.loc[k+1, "patient_id"] == pt_id):
            seq_list.append(vocab[data.loc[k, "Medical_code"]])
            if data.loc[k+1, "admission_date"] == dt:
                segment_list.append(dt_idx)
            else:
                dt_idx += 1
                segment_list.append(dt_idx)
            k += 1
            if k + 1 == data.shape[0]:
                break
        pkList.append([pt_id, label, seq_list, segment_list])
        if k + 1 == data.shape[0]:
            break
        k += 1

In [33]:
eligible_list = []
control_list = []

preprocess(eligible_list, eligible_data, 1)
preprocess(control_list, control_data, 0)

print(len(eligible_list))
len(control_list)

KeyboardInterrupt: 

In [ ]:
print(len(eligible_list[0][3]))

552


In [ ]:
train_e, test_e = train_test_split(eligible_list, test_size=0.2, random_state=32)
train_e, valid_e = train_test_split(train_e, test_size=0.1, random_state=32)

train_c, test_c = train_test_split(control_list, test_size=0.2, random_state=32)
train_c, valid_c = train_test_split(train_c, test_size=0.1, random_state=32)

train_f = train_e + train_c
valid_f = valid_e + valid_c
test_f  = test_e + test_c
print (len(train_f),len(test_f),len(valid_f))

2124 591 236


In [ ]:
# with open('./fineTuneData/CI_train.pkl', 'wb') as f:
#     pkl.dump(train_f, f)
# with open('./fineTuneData/CI_test.pkl', 'wb') as f:
#     pkl.dump(test_f, f)
# with open('./fineTuneData/CI_valid.pkl', 'wb') as f:
#     pkl.dump(valid_f, f)

In [ ]:
torch.__version__

'1.11.0+cu113'

In [ ]:
torch.cuda.get_arch_list()

['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86']